### Flow to work: High risk industry dataframe creation

This notebook takes IDBR industry data and flow to work Census 2011 data to create a table, ons-hotspot-prod.wip.idbr_census_flowtowork_lsoa_highindustry, which contains an estimate of the number of each high risk worker in each LSOA. 

<b>Datasets required:</b><br><br>
wf02ew_oa.csv: This can be downloaded from [nomis](https://www.nomisweb.co.uk/output/census/2011/wf02ew_oa.zip). This dataset contains data on the flow of workers from Work Zones to Output Areas.<p>
Output_Area_to_Lower_Layer_Super_Output_Area_to_Middle_Layer_Super_Output_Area_to_Local_Authority_District__December_2011__Lookup_in_England_and_Wales.csv: This can be downloaded from the ONS geoportal [here](https://geoportal.statistics.gov.uk/datasets/ons::output-area-to-lower-layer-super-output-area-to-middle-layer-super-output-area-to-local-authority-district-december-2011-lookup-in-england-and-wales/about). This contains information to aggregate Output Areas to LSOAs.

In [ ]:
#This script is to aggregate the number of residences working at high risk industries.
#the initial industry list includes meat and fish processing, textiles, care, warehousing, ready meals.
#please note that the list is living with more understanding on the covid-19 
#load libraries
import matplotlib.pyplot as plt
import geopandas as gpd
import pandas as pd
from janitor import clean_names
import numpy as np

In [ ]:
%%bigquery idbr_data
SELECT
    wz11, sic_cd, SUM(n_employed) as n_employed
FROM `ons-hotspot-prod.ingest_ons.idbr_2019_wz11`
GROUP BY wz11, sic_cd

In [ ]:
print(idbr_data.head())

In [ ]:
#high risk industry list
high_risk_industry = pd.DataFrame(
 [['meat_and_fish_processing', 101, 'Processing and preserving of meat and production of meat products'],
  ['meat_and_fish_processing', 102, 'Processing and preserving of fish, crustaceans and molluscs'],
  ['textiles', 131, 'Preparation and spinning of textile fibres'],
  ['textiles', 132, 'Weaving of textiles'],
  ['textiles', 133, 'Finishing of textiles'],
  ['textiles', 139, 'Manufacture of other textiles'],
  ['textiles', 141, 'Manufacture of wearing apparel, except fur apparel'],
  ['textiles', 142, 'Manufacture of articles of fur'],
  ['textiles', 143, 'Manufacture of knitted and crocheted apparel'],
  ['textiles', 151, 'Tanning and dressing of leather; manufacture of luggage, handbags, saddlery and harness; dressing and dyeing of fur'],
  ['textiles', 152, 'Manufacture of footwear'],
  ['care', 871, 'Residential nursing care activities'],
  ['care', 872, 'Residential care activities for learning disabilities, mental health and substance abuse'],
  ['care', 873, 'Residential care activities for the elderly and disabled'],
  ['care', 879, 'Other residential care activities'],
  ['warehousing', 521, 'Warehousing and storage'],
  ['ready_meals', 1085, 'Manufacture of prepared meals and dishes']], columns=['industry_name', 'sic_dig', 'sic_name'])

In [ ]:
#print(high_risk_industry)

In [ ]:
from tqdm import tqdm
import os.path

idbr_data['sic_cd'] = idbr_data['sic_cd'].astype(str)
high_risk_industry['sic_dig'] = high_risk_industry['sic_dig'].astype(str)

high_risk_industry_groupbyindustry = high_risk_industry.groupby(['industry_name'], as_index=False)

lsoa11_total_dataframe = pd.DataFrame(columns=['LSOA11CD'])

for high_risk_industry_name, high_risk_industry_group in high_risk_industry_groupbyindustry:
        
    print('*** ', high_risk_industry_name)
    print(high_risk_industry_group)
      
    #as the script was stopped from time to time; so load the previous industry if done to avoid running the loop again and again
    oldindustryfilepath = f'output_py/lsoa11_{high_risk_industry_name}_py.csv'
    oldtotalfilepath = f'output_py/lsoa11_high_risk_industry_py.csv'
    if os.path.isfile(oldindustryfilepath):
        print (f"{oldindustryfilepath} exist")
        lsoa11_total_dataframe = pd.read_csv(oldtotalfilepath)
        print(f"{oldtotalfilepath} loaded")
        print(lsoa11_total_dataframe.head())
        continue
        
    searchfor = (high_risk_industry_group['sic_dig'].values)
    #get the sub-industry table
    
    #idbr_data_highriskindustry = idbr_data[idbr_data['sic_cd'].str.contains('|'.join(searchfor))]
    print('*** searchfor: ', searchfor)
    idbr_data_highriskindustry = idbr_data[idbr_data['sic_cd'].str.startswith(tuple(searchfor))]
    print(idbr_data_highriskindustry.head())
    
    idbr_data_highriskindustry.to_csv(f'output_py/idbr_{high_risk_industry_name}.csv')
  
    industry_IDBR_2019_group = idbr_data_highriskindustry.groupby('wz11', as_index=False).sum()
    
    #This need replace with the journey to work dataset from Bigquery
    wf02ew = pd.read_csv("wf02ew_oa.csv") 
    wf02ew = clean_names(wf02ew)#, case="screaming_snake")
    
    idbr_wz_flows = wf02ew[wf02ew['area_of_workplace'].isin(idbr_data_highriskindustry['wz11'].values)]
    idbr_wz_flows_group = idbr_wz_flows.groupby('area_of_workplace', as_index=False)
    output_count_dataframe = pd.DataFrame(columns=['area_of_workplace', 'area_of_residence', 'count', 'groupprop', 'idbr_count'])


    for groupname, group in tqdm(idbr_wz_flows_group):
        area_of_workplace = groupname
        census11_groupsum = group['count'].sum()
    
        idbr_groupsum = industry_IDBR_2019_group[industry_IDBR_2019_group['wz11']==area_of_workplace]['n_employed'].iloc[0]

        outputgroup = group.copy()
        outputgroup['groupprop'] = group['count']/census11_groupsum
        outputgroup['idbr_count'] = outputgroup['groupprop']*idbr_groupsum
        output_count_dataframe = output_count_dataframe.append(outputgroup)

    
    output_count_dataframe.to_csv(f'output_py/count_{high_risk_industry_name}_py.csv')

    #calculate the number of residences living at area_of_residence working at meat_fish_poultry (high_risk industry)
    oa11_dataframe = output_count_dataframe.groupby('area_of_residence', as_index=False).sum()
    oa11_dataframe = pd.DataFrame({'AREA_OF_RESIDENCE':oa11_dataframe['area_of_residence'], 'N_HIGH_RISK': oa11_dataframe['idbr_count']})

    oa11_dataframe.to_csv(f'output_py/oa11_{high_risk_industry_name}_py.csv')


    #aggregate the data from OA to LSOA level.
    #load output area->LSOA lookup table
    oa_lookup = pd.read_csv("Output_Area_to_Lower_Layer_Super_Output_Area_to_Middle_Layer_Super_Output_Area_to_Local_Authority_District__December_2011__Lookup_in_England_and_Wales.csv")

    #output LSOA11 level count
    lsoa11_dataframe = pd.merge(oa11_dataframe, oa_lookup, left_on = 'AREA_OF_RESIDENCE', right_on = 'OA11CD', how='left')
    lsoa11_dataframe = lsoa11_dataframe.groupby('LSOA11CD', as_index=False).sum()
    lsoa11_dataframe = pd.DataFrame({'LSOA11CD': lsoa11_dataframe['LSOA11CD'], high_risk_industry_name: lsoa11_dataframe['N_HIGH_RISK']})
    lsoa11_dataframe.to_csv(f'output_py/lsoa11_{high_risk_industry_name}_py.csv')
    
    if (lsoa11_total_dataframe.empty):
        lsoa11_total_dataframe = lsoa11_dataframe
    else:
        lsoa11_total_dataframe = pd.merge(lsoa11_total_dataframe, lsoa11_dataframe, on='LSOA11CD', how='outer')
    lsoa11_total_dataframe.to_csv(f'output_py/lsoa11_high_risk_industry_py.csv', index=False)

    
lsoa11_total_dataframe.to_csv(f'output_py/lsoa11_high_risk_industry_py.csv', index=False)



    
    

In [ ]:
print(lsoa11_total_dataframe)

In [ ]:
#load to wip
from google.cloud import bigquery
client = bigquery.Client()
table_id = 'ons-hotspot-prod.wip.idbr_census_flowtowork_lsoa_highindustry'
#df = lsoa11_total_dataframe
df = pd.read_csv(f'output_py/lsoa11_high_risk_industry_py.csv', index_col=0)
print(df)
job_config = bigquery.LoadJobConfig(
    write_disposition="WRITE_TRUNCATE",
)
job = client.load_table_from_dataframe(
    df, table_id, job_config=job_config
)  # Make an API request.

job.result()  # Wait for the job to complete.

In [ ]:
# two digit sic code to section code look up

twodigsic_section_content = [
    
[1, "Crop and animal production, hunting and related service activities", "A"],
[2, "Forestry and logging", "A"],
[3, "Fishing and aquaculture", "A"],
[5, "Mining of coal and lignite", "B"],
[6, "Extraction of crude petroleum and natural gas", "B"],
[7, "Mining of metal ores", "B"],
[8, "Other mining and quarrying", "B"],
[9, "Mining support service activities", "B"],
[10, "Manufacture of food products", "C"],
[11, "Manufacture of beverages", "C"],
[12, "Manufacture of tobacco products", "C"],
[13, "Manufacture of textiles", "C"],
[14, "Manufacture of wearing apparel", "C"],
[15, "Manufacture of leather and related products", "C"],
[16, "Manufacture of wood and of products of wood and cork, except furniture; manufacture of articles of straw and plaiting materials", "C"],
[17, "Manufacture of paper and paper products", "C"],
[18, "Printing and reproduction of recorded media", "C"],
[19, "Manufacture of coke and refined petroleum products", "C"],
[20, "Manufacture of chemicals and chemical products", "C"],
[21, "Manufacture of basic pharmaceutical products and pharmaceutical preparations", "C"],
[22, "Manufacture of rubber and plastic products" ,"C"],
[23, "Manufacture of other non-metallic mineral products", "C"],
[24, "Manufacture of basic metals", "C"],
[25, "Manufacture of fabricated metal products, except machinery and equipment", "C"],
[26, "Manufacture of computer, electronic and optical products", "C"],
[27, "Manufacture of electrical equipment", "C"],
[28, "Manufacture of machinery and equipment n.e.c.", "C"],
[29, "Manufacture of motor vehicles, trailers and semi-trailers", "C"],
[30, "Manufacture of other transport equipment", "C"],
[31, "Manufacture of furniture", "C"],
[32, "Other manufacturing", "C"],
[33, "Repair and installation of machinery and equipment", "C"],
[35, "Electricity, gas, steam and air conditioning supply", "D"],
[36, "Water collection, treatment and supply", "E"],
[37, "Sewerage", "E"],
[38, "Waste collection, treatment and disposal activities; materials recovery", "E"],
[39, "Remediation activities and other waste management services.", "E"],
[41, "Construction of buildings", "F"],
[42, "Civil engineering" ,"F"],
[43, "Specialised construction activities", "F"],
[45, "Wholesale and retail trade and repair of motor vehicles and motorcycles" ,"G"],
[46, "Wholesale trade, except of motor vehicles and motorcycles", "G"],
[47, "Retail trade, except of motor vehicles and motorcycles", "G"],
[49, "Land transport and transport via pipelines", "H"],
[50, "Water transport", "H"],
[51, "Air transport", "H"],
[52, "Warehousing and support activities for transportation", "H"],
[53, "Postal and courier activities", "H"],
[55, "Accommodation", "I"],
[56, "Food and beverage service activities", "I"],
[58, "Publishing activities", "J"],
[59, "Motion picture, video and television programme production, sound recording and music publishing activities", "J"],
[60, "Programming and broadcasting activities", "J"],
[61, "Telecommunications", "J"],
[62, "Computer programming, consultancy and related activities", "J"],
[63, "Information service activities", "J"],
[64, "Financial service activities, except insurance and pension funding", "K"],
[65, "Insurance, reinsurance and pension funding, except compulsory social security", "K"],
[66, "Activities auxiliary to financial services and insurance activities", "K"],
[68, "Real estate activities", "L"],
[69, "Legal and accounting activities", "M"],
[70, "Activities of head offices; management consultancy activities", "M"],
[71, "Architectural and engineering activities; technical testing and analysis", "M"],
[72, "Scientific research and development", "M"],
[73, "Advertising and market research", "M"],
[74, "Other professional, scientific and technical activities", "M"],
[75, "Veterinary activities", "M"],
[77, "Rental and leasing activities", "N"],
[78, "Employment activities", "N"],
[79, "Travel agency, tour operator and other reservation service and related activities", "N"],
[80, "Security and investigation activities", "N"],
[81, "Services to buildings and landscape activities", "N"],
[82, "Office administrative, office support and other business support activities", "N"],
[84, "Public administration and defence; compulsory social security", "O"],
[85, "Education", "P"],
[86, "Human health activities", "Q"],
[87, "Residential care activities", "Q"],
[88, "Social work activities without accommodation", "Q"],
[90, "Creative, arts and entertainment activities", "R"],
[91, "Libraries, archives, museums and other cultural activities", "R"],
[92, "Gambling and betting activities", "R"],
[93, "Sports activities and amusement and recreation activities", "R"],
[94, "Activities of membership organisations", "S"],
[95, "Repair of computers and personal and household goods", "S"],
[96, "Other personal service activities", "S"],
[97, "Activities of households as employers of domestic personnel", "T"],
[98, "Undifferentiated goods- and services-producing activities of private households for own use", "T"],
[99, "Activities of extraterritorial organisations and bodies", "U"]
    
]

twodigsic_section_lookup = pd.DataFrame(twodigsic_section_content, columns = ['two_dig_sic', 'description', 'section'])

In [ ]:
#merge IDBR to industry sections
idbr_data['sic_cd'] = idbr_data['sic_cd'].astype(str).apply(lambda x: x.zfill(5))
idbr_data['two_dig_sic'] = idbr_data.sic_cd.str[:2]
idbr_data['two_dig_sic'] = idbr_data['two_dig_sic'].astype(int)
idbr_data = pd.merge(idbr_data, twodigsic_section_lookup, on='two_dig_sic', how='left' )
#idbr_data.to_csv('temp.csv')

In [ ]:
#group by section
idbr_data_groupbysection = idbr_data.groupby(['wz11','section'], as_index=False).sum().drop(['two_dig_sic'], axis=1)

print(idbr_data_groupbysection)

In [ ]:
# section code to section name lookup

section_name_content = {'section': [
    
    #"A",
    "B",
    "C",
    "D",
    "E",
    "F",
    "G",
    "H",
    "I",
    "J",
    "K",
    "L",
    "M",
    "N",
    "O",
    "P",
    "Q",
    "R",
    "S",
    "T",
    "U"
    
], 
                   'section_name': [
                       
    #"AGRICULTURE, FORESTRY AND FISHING",
    "MINING AND QUARRYING",
    "MANUFACTURING",
    "ELECTRICITY, GAS, STEAM AND AIR CONDITIONING SUPPLY",
    "WATER SUPPLY; SEWERAGE, WASTE MANAGEMENT AND REMEDIATION ACTIVITIES",
    "CONSTRUCTION",
    "WHOLESALE AND RETAIL TRADE; REPAIR OF MOTOR VEHICLES AND MOTORCYCLES",
    "TRANSPORTATION AND STORAGE",
    "ACCOMMODATION AND FOOD SERVICE ACTIVITIES",
    "INFORMATION AND COMMUNICATION",
    "FINANCIAL AND INSURANCE ACTIVITIES",
    "REAL ESTATE ACTIVITIES",
    "PROFESSIONAL, SCIENTIFIC AND TECHNICAL ACTIVITIES",
    "ADMINISTRATIVE AND SUPPORT SERVICE ACTIVITIES",
    "PUBLIC ADMINISTRATION AND DEFENCE; COMPULSORY SOCIAL SECURITY",
    "EDUCATION", 
    "HUMAN HEALTH AND SOCIAL WORK ACTIVITIES",
    "ARTS, ENTERTAINMENT AND RECREATION", 
    "OTHER SERVICE ACTIVITIES",
    "ACTIVITIES OF HOUSEHOLDS AS EMPLOYERS; UNDIFFERENTIATED GOODS-AND SERVICES-PRODUCING ACTIVITIES OF HOUSEHOLDS FOR OWN USE",
    "ACTIVITIES OF EXTRATERRITORIAL ORGANISATIONS AND BODIES"
        
]}

section_name_data = pd.DataFrame(section_name_content)
section_name_data.head(1)

In [ ]:
#for index, row in section_name_data:
from tqdm import tqdm

if True:
    row = section_name_data.iloc[1]
    print(row)
    
    industry_section_cd = row['section']
    industry_name = row['section_name']
    print(industry_section_cd)
    idbr_data_onesection = idbr_data_groupbysection[idbr_data_groupbysection['section']==industry_section_cd]
    filename = row['section_name']+'.csv'
    idbr_data_onesection.to_csv(filename)
      
    # aggregate the number of employees by WZ11 code at this section
    industry_IDBR_2019_group = idbr_data_onesection.groupby('wz11', as_index=False).sum()
    
    wf02ew = pd.read_csv("wf02ew_oa.csv") 
    wf02ew = clean_names(wf02ew)#, case="screaming_snake")
    
    idbr_wz_flows = wf02ew[wf02ew['area_of_workplace'].isin(idbr_data_onesection['wz11'].values)]
    
    idbr_wz_flows_group = idbr_wz_flows.groupby('area_of_workplace', as_index=False)
    
    output_count_dataframe = pd.DataFrame(columns=['area_of_workplace', 'area_of_residence', 'count', 'groupprop', 'idbr_count'])
    
    for groupname, group in tqdm(idbr_wz_flows_group):
        #calculate the number of residences working at area_of_workplace (WZ11) from cenesus2011    
        area_of_workplace = groupname
        census11_groupsum = group['count'].sum()
    
        #calculate the number of employees working at area_of_workplace from IDBR2019
        idbr_groupsum = industry_IDBR_2019_group[industry_IDBR_2019_group['wz11']==area_of_workplace]['n_employed'].iloc[0]

        outputgroup = group.copy()
        outputgroup['groupprop'] = group['count']/census11_groupsum
        #work out the number of employees who live in different area_of residence and work at the area_of_workplace
        outputgroup['idbr_count'] = outputgroup['groupprop']*idbr_groupsum
    
        output_count_dataframe = output_count_dataframe.append(outputgroup)
        
    output_count_dataframe.to_csv(f'output_py/count_{industry_name}_py.csv')
    
    #calculate the number of residences living at area_of_residence working at meat_fish_poultry (high_risk industry)
    oa11_dataframe = output_count_dataframe.groupby('area_of_residence', as_index=False).sum()
    oa11_dataframe = pd.DataFrame({'AREA_OF_RESIDENCE':oa11_dataframe['area_of_residence'], 'N_HIGH_RISK': oa11_dataframe['idbr_count']})

    oa11_dataframe.to_csv(f'output_py/oa11_{industry_name}_py.csv')
    print(f'oa11 count is saved to output_py/oa11_{industry_name}_py.csv')


    #aggregate the data from OA to LSOA level.
    #load output area->LSOA lookup table
    oa_lookup = pd.read_csv("Output_Area_to_Lower_Layer_Super_Output_Area_to_Middle_Layer_Super_Output_Area_to_Local_Authority_District__December_2011__Lookup_in_England_and_Wales.csv")

    #output LSOA11 level count
    lsoa11_dataframe = pd.merge(oa11_dataframe, oa_lookup, left_on = 'AREA_OF_RESIDENCE', right_on = 'OA11CD', how='left')
    lsoa11_dataframe = lsoa11_dataframe.groupby('LSOA11CD', as_index=False).sum()
    lsoa11_dataframe = pd.DataFrame({'LSOA11CD': lsoa11_dataframe['LSOA11CD'], industry_section_cd: lsoa11_dataframe['N_HIGH_RISK']})
    lsoa11_dataframe.to_csv(f'output_py/lsoa11_{industry_name}_py.csv')

        
        